In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
########## directory and file generate testing ##########
# file_content = "Hello, this is a text file."


# my_file_path = "/content/drive/My Drive/test-folder/example10.txt"

# if not os.path.isdir("/content/drive/My Drive/test-folder"):
#   os.mkdir("/content/drive/My Drive/test-folder")

# with open(my_file_path, "w") as file:
#     file.write(file_content)

# print("File saved to Google Drive!")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import random

from utils import progress_bar
import resnet18k
import mcnn

In [ ]:
arg_lr = 0.0001
arg_resume = False
arg_batch_size = 128
arg_model = "resnet18k"
arg_optimizer = "adam"
arg_noise = 0.0 #0.0, 0.1, 0.2
arg_data_size = 1.0
arg_data = "cifar10"
arg_w_param = 2 #1,2,4,8,16,32,64
arg_epoch = 1000

main_path = f'/content/drive/My Drive/ml-advanced-result'


In [ ]:
main_path = f'/content/drive/My Drive/ml-advanced-result/{arg_noise}_{arg_w_param}'
#main_path = f'/content/drive/My Drive/ML_hw/{arg_model}_{arg_data}_{arg_optimizer}/noise-{arg_noise}_datasize-{arg_data_size}_w_param-{arg_w_param}'

device = 'cuda' if torch.cuda.is_available() else 'cpu'

best_acc = 0  # best test accuracy

start_epoch = 0  # start from epoch 0 or last checkpoint epoch

if arg_data == 'cifar10':
  num_classes = 10
elif arg_data == 'cifar100':
  num_classes = 100

In [ ]:
def prepare_dataset():
    # Data
    print('==> Preparing data..')
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    if arg_data == 'cifar10':
        trainset = torchvision.datasets.CIFAR10(
            root='./data', train=True, download=True, transform=transform_train)
        new_len = int(arg_data_size*len(trainset))
        if arg_data_size < 1.0:
            trainset, _ = torch.utils.data.random_split(trainset, [new_len, len(trainset)-new_len])
        if arg_noise > 0.0:
            noise_len = int(len(trainset) * arg_noise)
            index = torch.randperm(len(trainset))[:noise_len]
            for i in index:
                noise_label = (trainset.targets[i] - 9)*(-1)
                trainset.targets[i] = noise_label

        trainloader = torch.utils.data.DataLoader(
            trainset, batch_size=arg_batch_size, shuffle=True, num_workers=2)

        testset = torchvision.datasets.CIFAR10(
            root='./data', train=False, download=True, transform=transform_test)
        testloader = torch.utils.data.DataLoader(
            testset, batch_size=100, shuffle=False, num_workers=2)

    elif arg_data == 'cifar100':
        trainset = torchvision.datasets.CIFAR100(
            root='./data', train=True, download=True, transform=transform_train)
        new_len = int(arg_data_size*len(trainset))
        if arg_data_size < 1.0:
            trainset, _ = torch.utils.data.random_split(trainset, [new_len, len(trainset)-new_len])
        if arg_noise > 0.0:
            noise_len = int(len(trainset) * arg_noise)
            index = torch.randperm(len(trainset))[:noise_len]
            for i in index:
                noise_label = (trainset.targets[i] - 9)*(-1)
                trainset.targets[i] = noise_label

        trainloader = torch.utils.data.DataLoader(
            trainset, batch_size=arg_batch_size, shuffle=True, num_workers=2)

        testset = torchvision.datasets.CIFAR100(
            root='./data', train=False, download=True, transform=transform_test)
        testloader = torch.utils.data.DataLoader(
            testset, batch_size=100, shuffle=False, num_workers=2)

    return trainloader, testloader

prepare_dataset()

==> Preparing data..


100%|██████████| 170498071/170498071 [00:02<00:00, 77839617.72it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


(<torch.utils.data.dataloader.DataLoader at 0x78d4a3572170>,
 <torch.utils.data.dataloader.DataLoader at 0x78d4a3573400>)

In [ ]:
# Model
print('==> Building model..')
if arg_model == "resnet18k":
    net = resnet18k.make_resnet18k(k = arg_w_param, num_classes = num_classes)
    net = net.to(device)
elif arg_model == "mcnn":
    net = mcnn.make_cnn(c = arg_w_param, num_classes = num_classes)
    net = net.to(device)

if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

==> Building model..


In [ ]:
if arg_resume==True:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir(os.path.join(main_path, 'checkpoint')), 'Error: no checkpoint directory found!'
    checkpoint = torch.load(os.path.join(main_path,'./checkpoint/ckpt.pth'))
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

In [ ]:
criterion = nn.CrossEntropyLoss()
if arg_optimizer == "sgd":
    optimizer = optim.SGD(net.parameters(), lr=0.1, weight_decay=5e-4)
    epoch = arg_epoch
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
elif arg_optimizer == "adam":
    optimizer = optim.Adam(net.parameters(), lr=arg_lr)
    epoch = arg_epoch

In [ ]:
train_loss_history = []
train_accuracy_history = []
test_loss_history = []
test_accuracy_history = []

os.makedirs(main_path, exist_ok=True)

In [ ]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()


        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        train_accuracy = 100.*correct/total

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

    train_loss_history.append(train_loss) #여기
    train_accuracy_history.append(train_accuracy) #여기

In [ ]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            test_accuracy = 100.*correct/total

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        test_loss_history.append(test_loss) #여기
        test_accuracy_history.append(test_accuracy) #여기

    # Save checkpoint.
    acc = 100.*correct/total
    print('Saving..')
    state = {
        'net': net.state_dict(),
        'acc': acc,
        'epoch': epoch,
    }
    if not os.path.isdir(os.path.join(main_path,'checkpoint')):
        os.mkdir(os.path.join(main_path,'checkpoint'))
    torch.save(state, os.path.join(main_path,'./checkpoint/ckpt.pth'))
    best_acc = acc

In [ ]:
if __name__ == "__main__":
    trainloader, testloader = prepare_dataset()

    for epoch in range(start_epoch, start_epoch+epoch):
    #for epoch in range(100):
        train(epoch)
        test(epoch)
        if arg_optimizer=='sgd':
            scheduler.step()

        torch.save({
          'train_loss_history': train_loss_history,
          'train_accuracy_history': train_accuracy_history,
          'test_loss_history': test_loss_history,
          'test_accuracy_history': test_accuracy_history,
        }, os.path.join(main_path,f'history_{epoch}.pth'))






==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0


KeyboardInterrupt: 

In [ ]:
checkpoint_path = os.path.join(main_path, 'checkpoint/ckpt.pth')

# 모델 상태 로드
checkpoint = torch.load(checkpoint_path)
net.load_state_dict(checkpoint['net'])
my_acc = checkpoint['acc']
my_epoch = checkpoint['epoch']

for k in range(my_epoch + 1, my_epoch + 101):
        train(k)
        test(k)
        if arg_optimizer=='sgd':
            scheduler.step()

        torch.save({
          'train_loss_history': train_loss_history,
          'train_accuracy_history': train_accuracy_history,
          'test_loss_history': test_loss_history,
          'test_accuracy_history': test_accuracy_history,
        }, os.path.join(main_path,f'history_{k}.pth'))



RuntimeError: Error(s) in loading state_dict for PreActResNet:
	Missing key(s) in state_dict: "conv1.weight", "layer1.0.bn1.weight", "layer1.0.bn1.bias", "layer1.0.bn1.running_mean", "layer1.0.bn1.running_var", "layer1.0.conv1.weight", "layer1.0.bn2.weight", "layer1.0.bn2.bias", "layer1.0.bn2.running_mean", "layer1.0.bn2.running_var", "layer1.0.conv2.weight", "layer1.1.bn1.weight", "layer1.1.bn1.bias", "layer1.1.bn1.running_mean", "layer1.1.bn1.running_var", "layer1.1.conv1.weight", "layer1.1.bn2.weight", "layer1.1.bn2.bias", "layer1.1.bn2.running_mean", "layer1.1.bn2.running_var", "layer1.1.conv2.weight", "layer2.0.bn1.weight", "layer2.0.bn1.bias", "layer2.0.bn1.running_mean", "layer2.0.bn1.running_var", "layer2.0.conv1.weight", "layer2.0.bn2.weight", "layer2.0.bn2.bias", "layer2.0.bn2.running_mean", "layer2.0.bn2.running_var", "layer2.0.conv2.weight", "layer2.0.shortcut.0.weight", "layer2.1.bn1.weight", "layer2.1.bn1.bias", "layer2.1.bn1.running_mean", "layer2.1.bn1.running_var", "la